## Step 0: Defining parameters
Please modify the parameters as follows:
1. `country` that you are working on.

2. `mode` can be one of two values "9to5" or "overnight". Don't forget to use double quotes.

Most of you will only use the "9to5" value. Which will tell the code to only translate 1,000 articles. This amount of articles takes between 3 and 4 hours to run. So this is ideal to use when you are in the office. Some of us (I'm looking at you DAU), will leave the code running the whole night to translate 4,000 articles every night. You are welcome to use the "overnight" mode if you want to finish your batch quickly (although we are not expecting you to finish it... unless you are an active member of the DAU).

3. `counter_day` and `counter_night` are just a counter of how many times you have SUCESSFULLY executed the code using the "9to5" or the "overnight" mode, respectively. If this is the first time you are running the code, both counters should be zero. If you already SUCESSFULLY run the code once using the "9to5" mode, then counter_day should be equal to one. If the code stopped or you had an issue and the code did not finished running, then it DOES NOT count as a SUCESSFUL execution.

4. `folow_up` can take two values True or False. Set it to True if you already finished a translation batch and your session is still running (and your master data is already uploaded). This way you don't need to execute all cells again, only the ones required. If this is the first time you are running this script during your current session, then set this value to False. No double quotes needed.

In [254]:
country       = "Hungary"
mode          = "9to5" # One of two values: "9to5" OR "overnight"
counter_day   = 2
counter_night = 0
follow_up     = False
suffix        = "retrans" # One of two values: "tp" OR "retrans"

## Step 1: Installing and importing required libraries

In [255]:
# if not follow_up:
#   !pip install nltk
#   !pip install deep_translator

In [256]:
if not follow_up:
  import os
  import pandas as pd
  import nltk
  from nltk.tokenize import sent_tokenize
  from deep_translator import GoogleTranslator
  nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ctoruno\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Step 2: Defining functions

In [257]:
def trans2english(text, sourcelang):
    """
    This functions gathers a text in a specific language and it returns its equivalent
    in English using the Google translation engine.

    Parameters:
        text:       String. Text to translate.
        sourcelang: String. Code of the source language you want to translate the text from.
    """
    if text:
      try:
          sentences = sent_tokenize(text)
          batch  = GoogleTranslator(source = sourcelang, target = "en").translate_batch(sentences)
          result = " ".join(batch)
          return result
      except Exception as e:
          out = f"Translation through API failed. Reason: {e}"
          return out
    else:
      return "No information available. No translation performed"

## Step 3: Reading the data

In [258]:
if not follow_up:
  master_data = pd.read_parquet(f"../data/data-extraction-1/data4translation/{country}_{suffix}.parquet.gzip")

## Step 4: Subsetting the data

In [259]:
batch = counter_day + counter_night + 1
if mode == "9to5":
    batch_size   = 500
    mode_counter = counter_day
if mode == "overnight":
    batch_size   = 2000
    mode_counter = counter_night

starting_row = (counter_day*500)+(counter_night*2000)
final_row    = starting_row+batch_size
batch_subset = master_data.copy().iloc[starting_row:final_row]

In [260]:
print(starting_row)
print(final_row)
print(master_data.shape)

1000
1500
(1384, 13)


In [261]:
batch_subset.head(3)

,id,link,domain_url,published_date,title,description,content,language,is_opinion,country,title_trans,description_trans,content_trans
958,4996577cb7214d7a6ebf5bac1515acfe,https://magyarnemzet.hu/kultura/2023/10/fekete...,magyarnemzet.hu,2023-10-24 03:17:00,Fekete Péter rangos cirkuszszakmai konferenciá...,A Kínai Wuqiao Nemzetközi Cirkuszfesztivál rés...,"– Rendkívüli megtiszteltetés számomra, hogy ez...",hu,False,Hungary,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...
962,477268131d6659639360b80814fe37a9,https://magyarnemzet.hu/belfold/2023/10/uj-kin...,magyarnemzet.hu,2023-10-23 14:43:15,Új kínai autógyár szemelte ki hazánkat?,A világ legnagyobb elektromos autógyártójára é...,A BYD már nem számít ismeretlen márkának Magya...,hu,False,Hungary,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...
970,962ce24c93d247fcbf93cbee2e12b381,https://magyarnemzet.hu/belfold/2023/10/gondoz...,magyarnemzet.hu,2023-10-22 19:02:11,Gondozatlan fővárosi temetők: Mártha Imréék el...,"Egy nyilatkozatból kiderült, hogy bár közeledi...",Ahogyan arról a \nMagyar Nemzet\n is beszámolt...,hu,False,Hungary,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...,Translation through API failed. Reason: HTTPSC...


## Step 5: Translating headline, description, and content

In [262]:
batch_subset[["title_trans", "description_trans", "content_trans"]] = batch_subset.apply(
    lambda row: row[["title", "description", "content"]].apply(lambda x: trans2english(text = x, sourcelang = row["language"])),
    axis = 1
)

## Step 6: Saving batch data

In [263]:
batch_subset.to_parquet(f"/users/ctoruno/Downloads/{country}_batch_{batch}_{mode}_{mode_counter}.parquet.gzip", compression = "gzip")

In [264]:
batch_subset["content_trans"].str.contains("API").value_counts()

content_trans
False    381
True       3
Name: count, dtype: int64